In [1]:
import os
os.chdir("..")

# Préparation 

## Imports

In [2]:
from pprint import pprint

In [3]:
import pandas as pd

pd.options.display.max_columns = 100

In [4]:
from src.utils import get_all_schema_path
from tableschema import Schema

## Lecture et mise en forme

In [5]:
csv_files = [
    "Variables_PMSI_HAD_181214170822.csv",
"Variables_PMSI_MCO_181214170820.csv",
"Variables_PMSI_PSY_181214170824.csv",
"Variables_PMSI_SSR_181214170823.csv"
]

In [6]:
df_list = []
for csv_name in csv_files:
    df = pd.read_csv("notebooks/Envoi_Dico Alimentation/" + csv_name, 
                 encoding="latin1", sep=';', dtype='str')
    columns = list(df.columns)
    df["produit"] = csv_name[10:18]
    
    df = df[["produit"] + columns]
    df_list.append(df)

df = pd.concat(df_list)

In [7]:
df.columns = ['produit', 
              'name_table', 
              'title_table', 
              'name', 
              'type', 
              'length',
              'description',
              "variable_jointure",
              "dateCreated",
              "dateDeleted",
              "observation_variable",
              'regle_gestion', 
              'famille_concept', 
              "date_created_table",
              "date_deleted_table",
              "sensible_ou_medicale"
             ]

In [8]:
df.head(2)

,produit,name_table,title_table,name,type,length,description,variable_jointure,dateCreated,dateDeleted,observation_variable,regle_gestion,famille_concept,date_created_table,date_deleted_table,sensible_ou_medicale
0,PMSI_HAD,T_HADaaA,Table des actes,ACT_COD,Caractère,1,Code de l'activité,NaN,2010,NaN,NaN,NaN,NaN,2010,NaN,NaN
1,PMSI_HAD,T_HADaaA,Table des actes,CCAM_COD,Caractère,7,Code de l'acte CCAM (hors extension PMSI),NaN,2010,NaN,NaN,NaN,NaN,2010,NaN,NaN


In [9]:
df.duplicated().sum()

6

In [10]:
df = df.drop_duplicates()

## Standardize values

In [11]:
def map_with_mapping(s):
    if s in mapping:
        return mapping[s]
    return s

### fillna et strip


In [12]:
df = df.fillna("")

In [13]:
for column in df.columns:
    df[column] = df[column].str.strip()

### name_table

In [14]:
df.name_table = df.name_table.str.upper()

### description

In [15]:
df.description = df.description.str.replace("", "’")

In [16]:
df.description = df.description.str.replace("", "œ")

### produit

In [17]:
df.produit.value_counts()

PMSI_MCO    1178
PMSI_SSR     608
PMSI_HAD     539
PMSI_PSY     372
Name: produit, dtype: int64

In [18]:
mapping = {
    'PMSI_MCO': 'PMSI MCO',
    'PMSI_SSR' : 'PMSI SSR',
    "PMSI_HAD" : "PMSI HAD",
    "PMSI_PSY" : "PMSI RIM-P"
}
df.produit = df.produit.map(lambda s: mapping[s] if s in mapping else s)
df.produit.value_counts()

PMSI MCO      1178
PMSI SSR       608
PMSI HAD       539
PMSI RIM-P     372
Name: produit, dtype: int64

### type

In [19]:
df.type.value_counts()

Caractère    1887
Numérique     774
Date           20
               12
FAUX            2
date            2
Name: type, dtype: int64

In [20]:
df[df.type == "FAUX"]

,produit,name_table,title_table,name,type,length,description,variable_jointure,dateCreated,dateDeleted,observation_variable,regle_gestion,famille_concept,date_created_table,date_deleted_table,sensible_ou_medicale
99,PMSI RIM-P,T_RIPAAC,chainage,EXE_SOI_DTD,FAUX,0,date d'entrée,,2009,,,,,2007,,
100,PMSI RIM-P,T_RIPAAC,chainage,EXE_SOI_DTF,FAUX,0,date de sortie,,2009,,,,,2007,,


In [21]:
df.loc[df.name == "ETA_NUM_JUR", "type"] = 'string' # Numero finess

In [22]:
df.loc[df.name == "ETA_NUM_PMSI", "type"] = 'string' # Numero finess e-pmsi

In [23]:
df.loc[df.name == "ENT_AM", "type"] = "date" # date ou string dans les autres tables

In [24]:
df.loc[df.name == "SOR_AM", "type"] = "date" # date ou string dans les autres tables

In [25]:
df.loc[df.name == "UCD_NBR_ETH", "type"] = "number"
df.loc[df.name == "UCD_NBR_TOT", "type"] = "number"

In [26]:
df.loc[df.name == "ANN_IVG_PREC", "type"] = "string" # on conserve l'ancienne valeur et la même pour totue les tables

In [27]:
df.loc[df.name == "ACH_PRI_DER", "type"] = "number" # prix
df.loc[df.name == "MOY_PRI", "type"] = "number" # prix

In [28]:
df[df.name == "ANN_IVG_PREC"] 

,produit,name_table,title_table,name,type,length,description,variable_jointure,dateCreated,dateDeleted,observation_variable,regle_gestion,famille_concept,date_created_table,date_deleted_table,sensible_ou_medicale
341,PMSI MCO,T_MCOAAIVG,Fich comp IVG,ANN_IVG_PREC,string,4,Année de la dernière IVG,,2017,,Observations 2017 : cette variable figurait ju...,,,2017,,
1170,PMSI MCO,T_MCOAAB,Description du Séjour,ANN_IVG_PREC,string,4,Année de l’IVG précédente,,2015,2017,Observations 2017 : cette variable a été tranf...,,,2005,,


In [29]:
mapping = {
    "Numérique": "number",
    "Caractère": "string",
    "Date": "date",
    "numérique": "number",
    "caractère": "string",
    "VARCHAR2": "string",
    "Varchar": "string",
    "date": "date",
    "FAUX": "datetime"
}
df.type = df.type.map(lambda s: mapping[s] if s in mapping else s)

df.type.value_counts()

string      1890
number       777
date          28
datetime       2
Name: type, dtype: int64

### lenght

In [30]:
mask = (df.length.str.contains('\.') | df.length.str.contains('\,'))
df[mask].head(2)

,produit,name_table,title_table,name,type,length,description,variable_jointure,dateCreated,dateDeleted,observation_variable,regle_gestion,famille_concept,date_created_table,date_deleted_table,sensible_ou_medicale
144,PMSI MCO,T_MCOAADMIP,Fich comp dmi en sus,NBR_POS_PRI,number,"10,3",Prix d'achat multiplié par le nombre posé,NON,2008,,,,,2008,,non
164,PMSI MCO,T_MCOAAFA,OQN Entete facture,HON_AM_MNR,number,"8,2",Total honoraire remboursable AM,NON,2005,,Observations 2017 : Attention les montants son...,Utilisable par ETA_NUM||RSA_NUM,,2005,,non


In [31]:
df.loc[(df.name == "nbligne") & (df.length == "20"), "length"] = "10"

In [32]:
df.loc[(df.length == "0"), "length"] = None

In [33]:
df.loc[(df.length == ""), "length"] = None

In [34]:
df.length = df.length.str.replace('.', ',').str.strip(',')

In [45]:
df.loc[df.length.str.contains(",") & (df.type == "string"), "type"] = "number"

## Create descriptor

In [35]:
df[df.name_table.str.startswith("T_SUP")]

,produit,name_table,title_table,name,type,length,description,variable_jointure,dateCreated,dateDeleted,observation_variable,regle_gestion,famille_concept,date_created_table,date_deleted_table,sensible_ou_medicale
1086,PMSI MCO,T_SUPAAALD,Alternatives à la dialyse (extension de fichie...,ANN_MOI,date,None,Mois et année,,2007,2009,,,,2007,2009,
1087,PMSI MCO,T_SUPAAALD,Alternatives à la dialyse (extension de fichie...,ETA_NUM,string,9,Numéro FINESS e-PMSI,,2007,2009,,,,2007,2009,
1088,PMSI MCO,T_SUPAAALD,Alternatives à la dialyse (extension de fichie...,FIC_TYP,number,3,Type de fichier,,2007,2009,,,,2007,2009,
1089,PMSI MCO,T_SUPAAALD,Alternatives à la dialyse (extension de fichie...,NBR_REA_EAL,number,8,Nombre réalisé en activité libérale,,2007,2009,,,,2007,2009,
1090,PMSI MCO,T_SUPAAALD,Alternatives à la dialyse (extension de fichie...,NBR_REA_HAL,number,8,Nombre réalisé hors activité libérale,,2007,2009,,,,2007,2009,
1091,PMSI MCO,T_SUPAAALD,Alternatives à la dialyse (extension de fichie...,PRS_COD,string,9,Code Prestation,,2007,2009,,,,2007,2009,
1092,PMSI MCO,T_SUPAAATU,Passage aux urgences (extension de fichier .atu),ANN_MOI,date,None,Mois et année,NON,2007,2009,,,,2007,2009,non
1093,PMSI MCO,T_SUPAAATU,Passage aux urgences (extension de fichier .atu),AUT_TYP,string,6,Type d'autorisation,NON,2007,2009,Observations 2011 : Code : AUT_TYP : POSU >>P...,,,2007,2009,non
1094,PMSI MCO,T_SUPAAATU,Passage aux urgences (extension de fichier .atu),ETA_NUM,string,9,Numéro FINESS e-PMSI,OUI,2007,2009,,,,2007,2009,non
1095,PMSI MCO,T_SUPAAATU,Passage aux urgences (extension de fichier .atu),ETA_NUM_GEO,string,9,Premier numéro FINESS géographique,NON,2007,2009,,,,2007,2009,non


In [36]:
df.name_table.str[:7].value_counts()

T_MCOAA    1126
T_SSRAA     608
T_HADAA     539
T_RIPAA     372
T_SUPAA      52
Name: name_table, dtype: int64

In [37]:
df.name_table = df.name_table.str[:5] + 'aa_nn' + df.name_table.str[7:]

In [38]:
def get_field_descriptor(df, name, columns):
    return (df[df.name.str.upper() == name.upper()][columns]
            .to_dict(orient="records")[0])

In [48]:
for i, (produit, name_table) in df[["produit", 'name_table']].drop_duplicates().iterrows():
    # Restriction table
    sdf = df[(df.produit == produit) & (df.name_table == name_table)]
    
    assert sdf.title_table.nunique() == 1
    title = sdf.title_table.unique()[0]
    
    assert sdf.date_created_table.nunique() == 1
    date_created_table = sdf.date_created_table.unique()[0]

    assert sdf.date_deleted_table.nunique() == 1
    date_deleted_table = sdf.date_deleted_table.unique()[0]
   
    table_descriptor = {
        "name": name_table, 
        "title": title, 
        "produit": produit,
        "history": {
            "dateCreated": date_created_table,
            "dateDeleted": date_deleted_table,
        }
    }
    
   
    schema_path = "schemas/PMSI/{}/{}.json".format(produit, name_table)
    print()
    print(produit, name_table, title) 

    print(schema_path)
    if not os.path.exists(schema_path):
        print("- Creating schema")
        table_descriptor["history"]["dateMissing"] = []
        table_descriptor["description"] = ""
        schema = Schema(table_descriptor)
    else :
        schema = Schema(schema_path)
        table_descriptor["history"]["dateMissing"] = schema.descriptor["history"]["dateMissing"]
        schema.descriptor.update(table_descriptor)

    
    
    # Différences d'ensembles de variables
    schema_names = set([name.upper() for name in schema.field_names]) 
    dico_names = set(sdf.name.str.upper())
    set_dico_schema = dico_names - schema_names

    if set_dico_schema:
        for field in set_dico_schema:
            print("- Variable absente dans le schéma et ajoutée automatiquement", field)
            columns_to_use = ['name', 'description', 'type', 'length', 
                              'dateCreated', 'dateDeleted']
            field_descriptor = get_field_descriptor(sdf, field, columns_to_use)
            field_descriptor["nomenclature"] = "-"
            field_descriptor["dateMissing"] = []
            schema.add_field(field_descriptor).name
    
            
    # MAJ variables
    for field in schema.descriptor['fields']:
        name = field['name']
        if not name.upper() in dico_names:
            print('- Variable absente dans le dico cnam', name)
            continue

        columns_to_update = ['description', 'type', 'length', "dateCreated", 'dateDeleted']
        #columns_to_update = ['dateCreated']
        field_descriptor = get_field_descriptor(sdf, name, columns_to_update)
        for key in columns_to_update:
            if field_descriptor[key] == "" or field_descriptor[key] is None:
                field_descriptor.pop(key)
        field.update(field_descriptor)
    
    try:
        schema.commit(strict=True)
    except Exception as e:
        print(e.errors)
        raise e
    
    schema.save(schema_path, ensure_ascii=False)


PMSI HAD T_HADaa_nnA Table des actes
schemas/PMSI/PMSI HAD/T_HADaa_nnA.json

PMSI HAD T_HADaa_nnB Table des Résumés Anonyme Par Sous-Séquence (RAPSS)
schemas/PMSI/PMSI HAD/T_HADaa_nnB.json
- Variable absente dans le dico cnam ETA_NUM_ESMS

PMSI HAD T_HADaa_nnC Table des NIR foinisé deux fois et date entrèe/sortie complètes
schemas/PMSI/PMSI HAD/T_HADaa_nnC.json

PMSI HAD T_HADaa_nnD Table des diagnostics associés
schemas/PMSI/PMSI HAD/T_HADaa_nnD.json

PMSI HAD T_HADaa_nnDMPA Table des diagnostics liés au MPA (mode de prise en charge associé)
schemas/PMSI/PMSI HAD/T_HADaa_nnDMPA.json

PMSI HAD T_HADaa_nnDMPP Table des diagnostics liés au MPP (mode de prise en charge principal)
schemas/PMSI/PMSI HAD/T_HADaa_nnDMPP.json

PMSI HAD T_HADaa_nnE Table des Etablissements
schemas/PMSI/PMSI HAD/T_HADaa_nnE.json

PMSI HAD T_HADaa_nnEHPA Table des conventions HAD-EHPA
schemas/PMSI/PMSI HAD/T_HADaa_nnEHPA.json
- Variable absente dans le dico cnam ETA_NUM_ESMS

PMSI HAD T_HADaa_nnFA TableFA : Tabl


PMSI MCO T_MCOaa_nnMED Fich comp medicament en sus
schemas/PMSI/PMSI MCO/T_MCOaa_nnMED.json
- Variable absente dans le dico cnam COD_LES

PMSI MCO T_MCOaa_nnMEDATU Fich comp médicament soumis à autorisation temporaire d'utilisation
schemas/PMSI/PMSI MCO/T_MCOaa_nnMEDATU.json

PMSI MCO T_MCOaa_nnMEDTHROMBO Fich comp Méd. thrombolytiques pour le traitement des AVC ischémiques
schemas/PMSI/PMSI MCO/T_MCOaa_nnMEDTHROMBO.json

PMSI MCO T_MCOaa_nnPIE Fich comp prestation inter établissement
schemas/PMSI/PMSI MCO/T_MCOaa_nnPIE.json

PMSI MCO T_MCOaa_nnPORG Fich comp prélévement d'organe
schemas/PMSI/PMSI MCO/T_MCOaa_nnPORG.json

PMSI MCO T_MCOaa_nnSTC Prise en charge
schemas/PMSI/PMSI MCO/T_MCOaa_nnSTC.json
- Variable absente dans le dico cnam RAC_MNT_PAT_CTL
- Variable absente dans le dico cnam NBR_REJET_AMO_CTL
- Variable absente dans le dico cnam FAC_AMO_DT_CTL
- Variable absente dans le dico cnam FAC_AMC_DT_CTL
- Variable absente dans le dico cnam FAC_RAC_DT_CTL
- Variable absente dans l


PMSI MCO T_MCOaa_nnSUP_USMP Fich sup Médicaments dispensés en USMP (Unité Sanitaire en Milieu Pénitentiaire)
schemas/PMSI/PMSI MCO/T_MCOaa_nnSUP_USMP.json
- Creating schema
- Variable absente dans le schéma et ajoutée automatiquement ETA_NUM
- Variable absente dans le schéma et ajoutée automatiquement MNT_TOT_TTC
- Variable absente dans le schéma et ajoutée automatiquement FIC_TYP
- Variable absente dans le schéma et ajoutée automatiquement UCD_COD_7_13
- Variable absente dans le schéma et ajoutée automatiquement ANN_MOI

PMSI MCO T_MCOaa_nnSUP_SMUR Fich sup SMUR
schemas/PMSI/PMSI MCO/T_MCOaa_nnSUP_SMUR.json
- Creating schema
- Variable absente dans le schéma et ajoutée automatiquement ETA_NUM
- Variable absente dans le schéma et ajoutée automatiquement NBR_SOR_PRIM
- Variable absente dans le schéma et ajoutée automatiquement ETA_NUM_SMUR
- Variable absente dans le schéma et ajoutée automatiquement NBR_SOR_SND
- Variable absente dans le schéma et ajoutée automatiquement ANN_MOI
- Vari


PMSI MCO T_SUPaa_nnATU Passage aux urgences (extension de fichier .atu)
schemas/PMSI/PMSI MCO/T_SUPaa_nnATU.json
- Creating schema
- Variable absente dans le schéma et ajoutée automatiquement ETA_NUM
- Variable absente dans le schéma et ajoutée automatiquement ETA_NUM_GEO
- Variable absente dans le schéma et ajoutée automatiquement NBR_PAS
- Variable absente dans le schéma et ajoutée automatiquement FIC_TYP
- Variable absente dans le schéma et ajoutée automatiquement NBR_HOS
- Variable absente dans le schéma et ajoutée automatiquement ANN_MOI
- Variable absente dans le schéma et ajoutée automatiquement AUT_TYP

PMSI MCO T_SUPaa_nnBPHN B et P hors nomenclature (extension de fichier .bphn)
schemas/PMSI/PMSI MCO/T_SUPaa_nnBPHN.json
- Creating schema
- Variable absente dans le schéma et ajoutée automatiquement NBR_FAC_AUT_ETP
- Variable absente dans le schéma et ajoutée automatiquement NBR_FAC_TOT
- Variable absente dans le schéma et ajoutée automatiquement ACT_COD
- Variable absente dans


PMSI RIM-P T_RIPaa_nnFA debut facture
schemas/PMSI/PMSI RIM-P/T_RIPaa_nnFA.json

PMSI RIM-P T_RIPaa_nnFB prestation hospitalière
schemas/PMSI/PMSI RIM-P/T_RIPaa_nnFB.json

PMSI RIM-P T_RIPaa_nnFC honoraire
schemas/PMSI/PMSI RIM-P/T_RIPaa_nnFC.json

PMSI RIM-P T_RIPaa_nnFH medicament
schemas/PMSI/PMSI RIM-P/T_RIPaa_nnFH.json

PMSI RIM-P T_RIPaa_nnFI interruption de sejour
schemas/PMSI/PMSI RIM-P/T_RIPaa_nnFI.json

PMSI RIM-P T_RIPaa_nnFL actes de Biologie
schemas/PMSI/PMSI RIM-P/T_RIPaa_nnFL.json

PMSI RIM-P T_RIPaa_nnFM ccam
schemas/PMSI/PMSI RIM-P/T_RIPaa_nnFM.json

PMSI RIM-P T_RIPaa_nnFP prothese
schemas/PMSI/PMSI RIM-P/T_RIPaa_nnFP.json

PMSI SSR T_SSRaa_nnB Description du Séjour
schemas/PMSI/PMSI SSR/T_SSRaa_nnB.json
- Variable absente dans le dico cnam GMT_NUM
- Variable absente dans le dico cnam EXB_TOP
- Variable absente dans le dico cnam EXB_NBJ
- Variable absente dans le dico cnam EXH_NBJ
- Variable absente dans le dico cnam LIT_DEDIE
- Variable absente dans le dico cnam SCO


PMSI SSR T_SSRaa_nnGHJ table des Groupes de Morbidité Dominante
schemas/PMSI/PMSI SSR/T_SSRaa_nnGHJ.json
- Creating schema
- Variable absente dans le schéma et ajoutée automatiquement ETA_NUM
- Variable absente dans le schéma et ajoutée automatiquement GHJ_COD
- Variable absente dans le schéma et ajoutée automatiquement RHA_VER
- Variable absente dans le schéma et ajoutée automatiquement NBR_JOU
- Variable absente dans le schéma et ajoutée automatiquement RHA_NUM


In [49]:
1

1